## Kaggle Datasets download
---

## Kaggle Dataset Collection

This notebook queries and downloads **public Kaggle datasets** in `.csv` format.  
- API authentication is handled via the `KAGGLE_USERNAME` and `KAGGLE_KEY` environment variables.  
- Datasets are retrieved across multiple pages of the Kaggle API and filtered to include only **public datasets**.  
- Each dataset is downloaded and extracted into the local directory:  
  - `./kaggledatasets`  

⚠️ **Note:** The exact datasets retrieved depend on the **time of download** and Kaggle’s dataset availability. The `dataset_dfs10000.pkl` dataset is provided and may not be overwritter for reproducibility.

---


In [ ]:
import os

#INSERT KAGGLE LOGIN
os.environ['KAGGLE_USERNAME'] = '...'
os.environ['KAGGLE_KEY'] = '...'

import kaggle
import pandas as pd
os.getcwd()
dataset_directory = './kaggledatasets'

# creating a df that contains datasets
datasets_list_csv = []

for page_num in range(400, 420):
    datasets_page = kaggle.api.datasets_list(page=page_num, max_size=100000, filetype='csv')
    datasets_list_csv.extend(datasets_page)

datasets_list_df = pd.DataFrame(datasets_list_csv)
public_datasets = datasets_list_df[datasets_list_df['isPrivate'] == False]


print(public_datasets)

# downloading the datasets into path = './datasets'
for index, row in public_datasets.iterrows():
    owner_slug = row['ownerNameNullable'].replace(" ", "-").lower()
    dataset_slug = row['titleNullable'].replace(" ", "-").lower()
    dataset_name = f"{owner_slug}/{dataset_slug}"
    print(f"Downloading dataset: {dataset_name}")
    try:
        kaggle.api.dataset_download_files(dataset_name, path=dataset_directory, unzip=True)
    except Exception as e:
        print(f"Error downloading dataset {dataset_name}: {e}")

                                      subtitleNullable creatorNameNullable  \
0               Data from Spotify's API on Tame Impala     Jarred Priester   
1                                                         Ashwin Siddharth   
2                                                               Aravind MS   
3                  Turnover data for Survival Analysis         Aman Mittal   
4    Taking a Look at Each Tournament Teams Turnove...     Patrick Watkins   
..                                                 ...                 ...   
395                                                                lavanya   
396                                                              Mansi2820   
397                                                          NOVITA DELVIA   
398                                                          Ivana Felicia   
399  Determining what factors explain alcohol consu...           Ranji Raj   

    creatorUrlNullable  totalBytesNullable  \
0       jarredpri

### Deletion of directorys which accidently got downloaded

In [7]:
import shutil

# directory path to delete
directory_path = 'kaggledatasets/Dataset'

# Check if the directory exists
if os.path.exists(directory_path):
    shutil.rmtree(directory_path)
    print("Directory deleted successfully.")
else:
    print("Directory does not exist.")

Directory deleted successfully.
